In [2]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from Figure import Figure

In [3]:
def show_image(name,img,key=500):
    cv2.imshow(f'{name}',img)
    cv2.waitKey(key)
    cv2.destroyAllWindows()

def write_image(path,img):
    cv2.imwrite(path,img)

In [4]:
#replicar los pasos de los patrones de detección en la foto
#leemos la foto
path = "Patterns\green_square.jpg"
img = imageio.imread(path)
# si traducimos al espacio hsv en vez de meterle los espacios rgb la comparación is_similar funciona, y lo detecta como un cuadrado verde
fig = Figure("square", "green", (50, 85, 55),  np.array([40, 100, 50]), np.array([70, 255, 255]), 4) #cambiarlo a rango de color hsv


C:\Users\mtfra\AppData\Local\Temp\ipykernel_17104\23946259.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(path)


In [6]:
# Convertir a espacio HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Definir límites del verde
lower_green = np.array([40, 100, 50])  # Límite inferior del verde
upper_green = np.array([70, 255, 255])

# Crear máscara para el verde
mask = cv2.inRange(hsv, lower_green, upper_green)
result = cv2.bitwise_and(img, img, mask=mask)
show_image("result",result, 5000)
# Convertir a escala de grises y encontrar contornos
kernel = np.ones((5, 5), np.uint8) #Dimensiones del kernel 
eroded = cv2.erode(mask, kernel, iterations=2)
show_image("eroded", eroded)
contours, _ = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Procesar cada contorno
for contour in contours:
    # Aproximar el contorno para encontrar esquinas
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Calcular el número de vértices
    n_vertex = len(approx)
    #print("num vertices", n_vertex)
    if n_vertex == 4:
        # Calcular el color promedio dentro del contorno
        mask = np.zeros(eroded.shape, dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, -1)
        #show_image("mask",mask)
        #TODO ESTO ESTA EN HSV Y ESTOY COMPARANDO EN RGB DENTRO DE LA CLASE FIGURE
        mean_color = cv2.mean(img, mask=mask)[:3]  # Ignorar canal alfa si existe
        mean_color = tuple(map(int, mean_color))   # Convertir a enteros
        
        print(mean_color)
        # Comparar con la figura teórica, vemos aquí el is similar
        if fig.is_similar(mean_color, n_vertex):
            print("color medio", mean_color)
            print("La figura detectada es similar al modelo teórico.")
       
    



(34, 82, 61)
color medio (34, 82, 61)
La figura detectada es similar al modelo teórico.
(34, 89, 65)
color medio (34, 89, 65)
La figura detectada es similar al modelo teórico.
(35, 86, 65)
color medio (35, 86, 65)
La figura detectada es similar al modelo teórico.
(32, 92, 68)
color medio (32, 92, 68)
La figura detectada es similar al modelo teórico.
(35, 90, 68)
color medio (35, 90, 68)
La figura detectada es similar al modelo teórico.
(37, 92, 70)
color medio (37, 92, 70)
La figura detectada es similar al modelo teórico.
